In [55]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import logging

logging.getLogger('jieba').setLevel(logging.WARN)
logging.getLogger('fgclassifier').setLevel(logging.INFO)
logging.getLogger('fgclassifier.features').setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import os

os.chdir('..')

In [3]:
from fgclassifier.utils import read_data, get_dataset

X_train, y_train = read_data(get_dataset('train_en'), flavor=None)
X_train.shape

2018-12-06 18:20:58,838 [INFO] Reading /opt/storage/english_train.csv..


(8000,)

In [4]:
np.linspace(200, 5000, 10, dtype=int)

array([ 200,  733, 1266, 1800, 2333, 2866, 3400, 3933, 4466, 5000])

In [5]:
from sklearn.pipeline import Pipeline

from IPython.display import clear_output
from sklearn.model_selection import GridSearchCV

from fgclassifier.features import Count, Tfidf, SVD
from fgclassifier.baseline import Baseline, MultiOutputClassifier
from fgclassifier.classifiers import SGD_SVC

2018-12-06 18:21:00,468 [INFO] 'pattern' package not found; tag filters are not available for English


See the maximum features we can get when limiting max_df and min_df
to a reasonable limit.

In [6]:
model = Count(ngram_range=(1, 3), min_df=5, max_df=0.9)
Xt = model.fit_transform(X_train)

# maximum number of degree frequent
len(model.vocabulary_)

2018-12-06 18:21:21,122 [INFO] Vocab Size: 84427


84427

In [54]:
docfreq = np.sum(Xt != 0, axis=0)
docfreq.sort().reverse()
docfreq

AttributeError: 'NoneType' object has no attribute 'reverse'

In [47]:
points = [0, 0.25, 0.5, 0.75, 0.8, 0.9, 0.95, 0.97, 0.99, 0.999, 1]
quantile = np.quantile(docfreq, points).astype(int)
pd.DataFrame({
    'quantile': points,
    'df': quantile
})

,quantile,df
0,0.000,5
1,0.250,6
2,0.500,9
3,0.750,19
4,0.800,24
5,0.900,48
6,0.950,95
7,0.970,158
8,0.990,433
9,0.999,2362


In [39]:
cutoffs = np.linspace(1000, 10000, 10, dtype=int)
pd.DataFrame({
    'cutoff': cutoffs,
    'pct_vocab': [np.sum(docfreq > c) for c in cutoffs]
})

,cutoff,pct_vocab
0,1000,287
1,2000,113
2,3000,54
3,4000,30
4,5000,20
5,6000,8
6,7000,2
7,8000,0
8,9000,0
9,10000,0


In [ ]:
n_min = 

pipeline = Pipeline([
    ('vect', Count(max_features=100, ngram_range=(1, 5),
                   min_df=5, max_df=0.99)),
    ('tfidf', Tfidf()),
    ('svd', SVD(500)),
    ('clf', MutiOutpuClassifier(SGD_SVC))
], memory='data/pipeline_cache')


parameters = {
    'vect__max_features': np.linspace(500, 1000, 10, dtype=int)
}

with joblib.parallel_backend('threading', n_jobs=5):
    searcher = GridSearchCV(
        clf, parameters, cv=5,
        verbose=True,
        return_train_score=True
    )
    display(searcher)
    searcher.fit(X_train, y_train)